# 3.5 Colocando o Istio em execução

## Instalando o Istio (linux)

Para instalar a última versão do Istio, neste momento 1.7.3, você pode ir até a página [Getting Started](https://istio.io/latest/docs/setup/getting-started/#download) ou seguir as instruções abaixo:

In [1]:
curl -L https://istio.io/downloadIstio | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   102  100   102    0     0    104      0 --:--:-- --:--:-- --:--:--   104
100  4277  100  4277    0     0   3088      0  0:00:01  0:00:01 --:--:--  3088

Istio 1.7.3 Download Complete!

Istio has been successfully downloaded into the istio-1.7.3 folder on your system.

Next Steps:
See https://istio.io/latest/docs/setup/install/ to add Istio to your Kubernetes cluster.

To configure the istioctl client tool for your workstation,
add the /Users/anderson/Downloads/tools/istio-curso/istio-1.7.3/bin directory to your environment path variable with:
	 export PATH="$PATH:/Users/anderson/Downloads/tools/istio-curso/istio-1.7.3/bin"

Begin the Istio pre-installation check by running:
	 istioctl x precheck 

Need more information? Visit https://istio.io/latest/docs/setup/install/ 


Para usar o comando `istioctl`, que está no diretório `bin` do download, coloque-o na variável `PATH` ou copie o arquivo `bin/istioctl` para um diretório no seu `PATH`

## Opção 1: Copiar o arquivo para seu diretório de binários

In [ ]:
# Verificar onde está o executável do Istio
ls istio-1.7.3/bin
# Verificar o conteúdo da variável PATH
echo $PATH
# Vamos coloca-lo no diretório bin do usuário
[ -d "$HOME/bin" ] && echo "O diretorio $HOME/bin existe."
# Copiar o arquivo para o diretório bin
cp istio-1.7.3/bin/istioctl ~/bin
# Teste
istioctl --help

## Opção 2: Definir a variável PATH
Você pode copiar o comando em qualquer diretório ou mantê-lo e ajustar a variável `PATH` na sessão.

In [2]:
export PATH=$PWD/istio-1.7.3/bin:$PATH
echo $PATH
istioctl --help

/Users/anderson/Downloads/tools/istio-curso/istio-1.7.3/bin:/Users/anderson/miniconda3/envs/py3/bin:/Users/anderson/miniconda3/condabin:/Applications/Julia-1.5.app/Contents/Resources/julia/bin:/Users/anderson/.istioctl/bin:/Users/anderson/google-cloud-sdk/bin:/Users/anderson/.krew/bin:/Users/anderson/.jenv/shims:/Users/anderson/.jenv/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/go/bin:/usr/local/share/dotnet:/opt/X11/bin:~/.dotnet/tools:/Library/Apple/usr/bin:/Library/Frameworks/Mono.framework/Versions/Current/Commands:/Users/anderson/bin:/Users/anderson/bin/oracle-client:/usr/local/bin:/usr/local/sbin:~/bin:/Users/anderson/.dotnet/tools
Istio configuration command line utility for service operators to
debug and diagnose their Istio mesh.

Usage:
  istioctl [command]

Available Commands:
  analyze         Analyze Istio configuration and print validation messages
  authz           (authz is experimental. Use `istioctl experimental authz`)
  convert-ingress

## Acessando o k8s

Você já deve ter acessado o cluster em seções anteriores, mas vale a pena revisar:

In [1]:
export KUBECONFIG=~/.kube/config
kubectl config get-contexts
kubectl get nodes

CURRENT   NAME                   CLUSTER                AUTHINFO                                                      NAMESPACE
          backbone-educacional   backbone-educacional   clusterUser_COGNA_BACKBONE_EDUCACIONAL_backbone-educacional   
*         docker-desktop         docker-desktop         docker-desktop                                                
NAME             STATUS   ROLES    AGE     VERSION
docker-desktop   Ready    master   3m13s   v1.18.8


## O mínimo Istio

Desde a versão 1.6 o Istio é composto de uma única entrega chamada `istiod` e ela pode ser instalada com o comando abaix:

In [2]:
istioctl install --set profile=minimal 

Detected that your cluster does not support third party JWT authentication. Falling back to less secure first party JWT. See https://istio.io/docs/ops/best-practices/security/#configure-third-party-service-account-tokens for details.
✔ Istio core installed                                                          
✔ Istiod installed                                                              
✔ Installation complete                                                         

Vamos verificar o que foi instalado

In [3]:
# Obtendo os namespaces
kubectl get ns

# Verificando o namespace do Istio
kubectl get all -n istio-system

NAME              STATUS   AGE
default           Active   3m28s
istio-system      Active   13s
kube-node-lease   Active   3m29s
kube-public       Active   3m29s
kube-system       Active   3m30s
NAME                         READY   STATUS    RESTARTS   AGE
pod/istiod-fb4fbff6b-nvrj4   1/1     Running   0          12s

NAME             TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                                         AGE
service/istiod   ClusterIP   10.104.166.242   <none>        15010/TCP,15012/TCP,443/TCP,15014/TCP,853/TCP   11s

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/istiod   1/1     1            1           12s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/istiod-fb4fbff6b   1         1         1       12s

NAME                                         REFERENCE           TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
horizontalpodautoscaler.autoscaling/istiod   Deployment/istiod   <unknown>/80%  

## Ativando o Istio em um namespace
Para que o Istiod adicione um proxy no mesmo pod que o(s) seu(s) containers, você precisa marcar o namespace da seguinte forma:

In [4]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


Vamos verificar como ficou a configuração do namespace

In [5]:
kubectl describe ns/default

Name:         default
Labels:       istio-injection=enabled
Annotations:  <none>
Status:       Active

No resource quota.

No LimitRange resource.


O `istiod` irá monitorar os namespaces com esse `label` e adicionar ao pod um container de proxy. Vamos fazer deploy de uma aplicação exemplo para verificar esse comportamento.

## Criando uma aplicação demo

Vamos utilizar o helm para criar uma aplicação simples.

In [15]:
# Vamos criar um diretório para nossos exemplos
mkdir -p exemplos/simple-app

# Deployment
cat <<EOT > exemplos/simple-app/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: "1.0"
spec:
  replicas: 1
  selector:
    matchLabels:
        app: simple-app
        version: "1.0"
  template:
    metadata:
      labels:
        app: simple-app
        version: "1.0"
    spec:
      containers:
        - name: simple-app
          image: "nginx:stable"
          imagePullPolicy: IfNotPresent
          ports:
            - name: http
              containerPort: 80
              protocol: TCP
EOT

# Service
cat <<EOT > exemplos/simple-app/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: simple-app
  labels:
    app: simple-app
    version: "1.0"
spec:
  type: ClusterIP
  ports:
    - port: 80
      targetPort: http
      protocol: TCP
      name: http
  selector:
    app: simple-app
    version: "1.0"
EOT

In [16]:
ls -la exemplos/simple-app

total 16
drwxr-xr-x   4 anderson  staff  128 Oct 22 09:38 .
drwxr-xr-x  10 anderson  staff  320 Oct 22 09:33 ..
-rw-r--r--   1 anderson  staff  527 Oct 22 09:56 deployment.yaml
-rw-r--r--   1 anderson  staff  266 Oct 22 09:56 service.yaml


Criamos dois arquivos, o `deployment.yaml` e o `service.yaml` no diretório `exemplos/simple-app`, agora vamos instala-la no cluster.

Inspecione os arquivos e tente descobrir o que será instalado no cluster, uma dica, procure a pela imagem.

In [19]:
kubectl apply -f exemplos/simple-app

deployment.apps/simple-app created
service/simple-app created


Vamos acessar nossa aplicação, ela foi configurada para o tipo de serviço `ClusterIP`, o que significa que o acesso é interno, apenas entre os PODs do cluster, mas podemos acessá-la utilizando o comando `kubectl port-forwar`.

In [21]:
kubectl port-forward svc/simple-app 8000:80

Forwarding from 127.0.0.1:8000 -> 80
Forwarding from [::1]:8000 -> 80
Handling connection for 8000


Vá até o seu navegador e entre acesse a url: <http://localhost:8000>

Pronto, você tem acesso a sua aplicação como se estivesse sendo executada na sua máquina. Claro que o kubernetes está na sua máquina, mas isso funcionaria em qualquer kubernetes, local ou remoto.

Para interromper:
* No jupyter Lab ou notebook: Clique no icone <kbd>◾</kbd> (_Interrup the kernel_ na barra de ferramentas)
* No terminal: tecle <kbd>CTRL</kbd>+<kbd>C</kbd>

Verificando o que foi instalado.

In [22]:
kubectl get --show-labels all

NAME                              READY   STATUS    RESTARTS   AGE   LABELS
pod/simple-app-6f6cfc8554-6m6t2   2/2     Running   0          44s   app=simple-app,istio.io/rev=default,pod-template-hash=6f6cfc8554,security.istio.io/tlsMode=istio,service.istio.io/canonical-name=simple-app,service.istio.io/canonical-revision=1.0,version=1.0

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE     LABELS
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   7d15h   component=apiserver,provider=kubernetes
service/simple-app   ClusterIP   10.109.84.250   <none>        80/TCP    44s     app=simple-app,version=1.0

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE   LABELS
deployment.apps/simple-app   1/1     1            1           44s   app=simple-app,version=1.0

NAME                                    DESIRED   CURRENT   READY   AGE   LABELS
replicaset.apps/simple-app-6f6cfc8554   1         1         1       44s   app=simple-app,pod

Note que o pod/simple-app tem 2/2 na coluna pronto (Ready), isso significa que dois containers de dois estão ok, vamos verificar quem é o segundo container.

In [23]:
# Usaremos um dos labels do pod para encontra-lo
kubectl describe pod -l app=simple-app

Name:         simple-app-6f6cfc8554-6m6t2
Namespace:    default
Priority:     0
Node:         docker-desktop/192.168.65.3
Start Time:   Thu, 22 Oct 2020 09:56:59 -0300
Labels:       app=simple-app
              istio.io/rev=default
              pod-template-hash=6f6cfc8554
              security.istio.io/tlsMode=istio
              service.istio.io/canonical-name=simple-app
              service.istio.io/canonical-revision=1.0
              version=1.0
Annotations:  prometheus.io/path: /stats/prometheus
              prometheus.io/port: 15020
              prometheus.io/scrape: true
              sidecar.istio.io/status:
                {"version":"8e6e902b765af607513b28d284940ee1421e9a0d07698741693b2663c7161c11","initContainers":["istio-init"],"containers":["istio-proxy"]...
Status:       Running
IP:           10.1.0.118
IPs:
  IP:           10.1.0.118
Controlled By:  ReplicaSet/simple-app-6f6cfc8554
Init Containers:
  istio-init:
    Container ID:  docker://9a64ffe0dcc1dba44021d3ce3

É uma grande quantidade de informação, vamos procurar uma seção chamada `Containers` e nela a nossa aplicação `simple-app`.

Como você pode ver, a imagem desse container é `nginx`, com a tag `stable`, mais abaixo tem um segundo container `istio-proxy`, com a imagem `docker.io/istio/proxyv2` e tag representando a versão `1.7.3`.

Esse container não faz parte do [exemplos/simple-app/deployment.yaml](exemplos/simple-app/deployment.yaml), ele foi adicionado ao seu pod pelo `istiod`.

Caso você precise saber todos os _namespaces_ que tem a injeção do _conteiner_ _proxy_ do Istio ativado, basta executar o comando:

In [25]:
kubectl get ns -l istio-injection=enabled

NAME      STATUS   AGE
default   Active   7d15h


Até o momento, somente o namespace `default`, onde configuramos a marcação, retornou.

## Conclusão

Neste ponto temos o mínimo do Istio em execução no nosso cluster, mas com exceção de um container extra isso não significa muito.

Na próxima parte iremos explorar os recursos que esta instalação mínima do Istio pode oferecer.